In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim

In [2]:
train_embeddings = np.load("./embeddings/deepfashion2_train_embeddings.npy")
train_labels = pd.read_csv("./dataset/DeepFashion2/img_info_dataframes/train.csv")['category_id'].values
train_labels = train_labels - train_labels.min()

val_embeddings = np.load("./embeddings/deepfashion2_validation_embeddings.npy")
val_labels = pd.read_csv("./dataset/DeepFashion2/img_info_dataframes/validation.csv")['category_id'].values
val_labels = val_labels - val_labels.min()

print(f"Loaded train embeddings: {train_embeddings.shape}, labels: {train_labels.shape}")
print(f"Loaded validation embeddings: {val_embeddings.shape}, labels: {val_labels.shape}")

Loaded train embeddings: (312186, 2048), labels: (312186,)
Loaded validation embeddings: (52490, 2048), labels: (52490,)


In [3]:
train_labels = train_labels.astype(int)
val_labels = val_labels.astype(int)

print(f"Unique train labels: {np.unique(train_labels)}")
print(f"Unique validation labels: {np.unique(val_labels)}")

Unique train labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
Unique validation labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12]


In [4]:
class FashionDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = torch.tensor(embeddings, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

In [5]:
batch_size = 64

train_dataset = FashionDataset(train_embeddings, train_labels)
val_dataset = FashionDataset(val_embeddings, val_labels)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f"Train & validation data loaders ready!")

Train & validation data loaders ready!


In [ ]:
num_classes = len(set(train_labels))

class FashionClassifier(nn.Module):
    def __init__(self, input_size=2048, num_classes=num_classes):
        super(FashionClassifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.fc(x)
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {device}")
model = FashionClassifier().to(device)

print(f"Model initialized with {num_classes} fashion categories!")

Training on cuda
Model initialized with 13 fashion categories!


In [7]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 50
patience = 5
best_val_loss = float("inf")
epochs_without_improvement = 0

for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0
    
    train_loader_tqdm = tqdm(train_loader, desc=f"Training", leave=False)
    for embeddings, labels in train_loader_tqdm:
        embeddings, labels = embeddings.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(embeddings)

        if labels.max() >= num_classes or labels.min() < 0:
            print(f"ERROR: Labels out of range! Found {labels.min()} to {labels.max()}")
            exit()

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        train_loader_tqdm.set_postfix(loss=loss.item(), acc=correct / total)

    train_acc = correct / total

    model.eval()
    val_loss, correct, total = 0, 0, 0
    
    val_loader_tqdm = tqdm(val_loader, desc=f"Validation", leave=False)
    with torch.no_grad():
        for embeddings, labels in val_loader_tqdm:
            embeddings, labels = embeddings.to(device), labels.to(device)
            outputs = model(embeddings)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            val_loader_tqdm.set_postfix(loss=loss.item(), acc=correct / total)

    val_loss /= len(val_loader)
    val_acc = correct / total

    print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {total_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), "./best_fashion_classifier.pth")
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
        break

model.load_state_dict(torch.load("./best_fashion_classifier.pth"))
torch.save(model, "./fashion_classifier_full.pth")

Epoch [1/50] - Train Loss: 8783.0584, Train Acc: 0.3523, Val Loss: 1.6989, Val Acc: 0.3697


Epoch [2/50] - Train Loss: 8313.5487, Train Acc: 0.3743, Val Loss: 1.6490, Val Acc: 0.3843


Epoch [3/50] - Train Loss: 8129.2720, Train Acc: 0.3824, Val Loss: 1.6412, Val Acc: 0.3855


Epoch [4/50] - Train Loss: 8004.3218, Train Acc: 0.3894, Val Loss: 1.6200, Val Acc: 0.3905


Epoch [5/50] - Train Loss: 7912.8763, Train Acc: 0.3938, Val Loss: 1.6152, Val Acc: 0.3930


...

Epoch [16/50] - Train Loss: 7325.9512, Train Acc: 0.4229, Val Loss: 1.5702, Val Acc: 0.4039


Epoch [17/50] - Train Loss: 7292.9507, Train Acc: 0.4248, Val Loss: 1.5712, Val Acc: 0.4036


Epoch [18/50] - Train Loss: 7259.1066, Train Acc: 0.4262, Val Loss: 1.5691, Val Acc: 0.4054


Epoch [19/50] - Train Loss: 7222.2623, Train Acc: 0.4281, Val Loss: 1.5766, Val Acc: 0.4052


C:\Users\krish\AppData\Local\Temp\ipykernel_24496\2885187009.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("./best_fashion_classifier